# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('cities.csv')

city_df = pd.read_csv(csv_file)
city_df


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,3874787,Punta Arenas,CL,-53.1500,-70.9167,46.51,66,0,4.61
1,3421765,Nanortalik,GL,60.1432,-45.2371,33.60,64,49,1.25
2,3465010,Cunha,BR,-23.0744,-44.9597,63.82,99,100,1.28
3,5866583,Saint Paul Harbor,US,57.7900,-152.4072,38.32,91,94,10.60
4,2122090,Pevek,RU,69.7008,170.3133,7.88,92,100,6.60
...,...,...,...,...,...,...,...,...,...
555,3395042,Mazagão,BR,-0.1150,-51.2894,82.45,86,35,3.18
556,390054,Baft,IR,29.2340,56.6057,53.22,54,49,5.93
557,2108857,Gizo Government Station,SB,-8.1030,156.8419,85.03,69,37,6.96
558,240210,Birao,CF,10.2849,22.7882,73.89,63,68,0.92


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Clouds'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
111,2446796,Bilma,NE,18.6853,12.9164,76.87,18,2,3.56
130,57000,Hobyo,SO,5.3505,48.5268,80.80,68,5,0.31
139,6690296,Saint-Joseph,RE,-21.3667,55.6167,75.18,69,6,0.25
182,64814,Bandarbeyla,SO,9.4942,50.8122,76.96,66,3,3.80
221,1106643,Quatre Cocos,MU,-20.2078,57.7625,77.09,62,1,9.19
241,2249281,Linguère,SN,15.3953,-15.1193,84.42,48,0,4.61
263,1266049,Kota,IN,25.1833,75.8333,76.44,40,0,1.63
274,1254046,Tura,IN,25.5198,90.2201,79.38,64,0,2.21
286,3384937,Miracema do Tocantins,BR,-9.5618,-48.3967,77.14,47,1,1.88
304,286621,Salalah,OM,17.0151,54.0924,80.33,61,3,4.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
111,2446796,Bilma,NE,18.6853,12.9164,76.87,18,2,3.56,
130,57000,Hobyo,SO,5.3505,48.5268,80.80,68,5,0.31,
139,6690296,Saint-Joseph,RE,-21.3667,55.6167,75.18,69,6,0.25,
182,64814,Bandarbeyla,SO,9.4942,50.8122,76.96,66,3,3.80,
221,1106643,Quatre Cocos,MU,-20.2078,57.7625,77.09,62,1,9.19,
241,2249281,Linguère,SN,15.3953,-15.1193,84.42,48,0,4.61,
263,1266049,Kota,IN,25.1833,75.8333,76.44,40,0,1.63,
274,1254046,Tura,IN,25.5198,90.2201,79.38,64,0,2.21,
286,3384937,Miracema do Tocantins,BR,-9.5618,-48.3967,77.14,47,1,1.88,
304,286621,Salalah,OM,17.0151,54.0924,80.33,61,3,4.25,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
111,2446796,Bilma,NE,18.6853,12.9164,76.87,18,2,3.56,NaN
130,57000,Hobyo,SO,5.3505,48.5268,80.80,68,5,0.31,Osmani Hotel
139,6690296,Saint-Joseph,RE,-21.3667,55.6167,75.18,69,6,0.25,"""Plantation Bed and Breakfast"
182,64814,Bandarbeyla,SO,9.4942,50.8122,76.96,66,3,3.80,JABIR HOTEL
221,1106643,Quatre Cocos,MU,-20.2078,57.7625,77.09,62,1,9.19,Constance Belle Mare Plage Mauritius
241,2249281,Linguère,SN,15.3953,-15.1193,84.42,48,0,4.61,AYO Casablanca
263,1266049,Kota,IN,25.1833,75.8333,76.44,40,0,1.63,Country Inn & Suites by Radisson Kota
274,1254046,Tura,IN,25.5198,90.2201,79.38,64,0,2.21,Hotel C&C
286,3384937,Miracema do Tocantins,BR,-9.5618,-48.3967,77.14,47,1,1.88,Miracema Palace Hotel
304,286621,Salalah,OM,17.0151,54.0924,80.33,61,3,4.25,Salalah Gardens Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))